# **SET UP**

## Install Libraries

In [1]:
! pip install sentence-transformers
! pip install --q unstructured langchain
! pip install --q "unstructured[all-docs]"


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-file 0.1.22 requires pypdf<5.0.0,>=4.0.1, but you have pypdf 6.1.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-file 0.1.22 requires pypdf<5.0.0,>=4.0.1, but you have pypdf 6.1.0 which is incompatible.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.16.0 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incom

In [ ]:
! pip install langchain_community fastembed chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Constants

In [3]:
# Define the directory where your PDFs are stored
pdf_directory = "C:/Users/ili/Downloads/test_rag"
save_dir = pdf_directory

## **1. Extract Texts from PDFs**
use **PyPDFLoader** from LangChain_Community to extract textual data <br>
from **Multipple PDFs**

In [ ]:
# general
import os
import datetime

# Lancgain
from langchain_community.document_loaders import PyPDFLoader

# Get a list of all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# Initialize lists to hold pages from Nvidia and Tesla PDFs separately
nvidia_pages = []


# Iterate through each PDF file and load it
for pdf_file in pdf_files:
    file_path = os.path.join(pdf_directory, pdf_file)
    print(f"Processing file: {file_path}\n")

    # Load the PDF and split it into pages
    loader = PyPDFLoader(file_path=file_path)
    pages = loader.load()


    nvidia_pages.extend(pages)



Processing file: C:/Users/ili/Downloads/test_rag\Nvidia_Rev_by_Mkt_Qtrly_Trend_Q225.pdf

Processing file: C:/Users/ili/Downloads/test_rag\Tesla_Q2FY25-CFO-Commentary.pdf



In [6]:
# print out the first page of the first document for each category as an example
if nvidia_pages:
    print("=========================================")
    print("First page of the first Nvidia document:")
    print("=========================================\n")
    print(nvidia_pages[0].page_content)
else:
    print("No Nvidia pages found in the PDFs.")

First page of the first Nvidia document:

($ in millions) Q2 FY25 Q1 FY25 Q4 FY24 Q3 FY24 Q2 FY24 Q1 FY24 Q4 FY23 Q3 FY23
Data Center $26,272 $22,563 $18,404 $14,514 $10,323 $4,284 $3,616 $3,833
Gaming 2,880 2,647 2,865 2,856 2,486 2,240 1,831 1,574
Professional 
Visualization 454 427 463 416 379 295 226 200
Auto 346 329 281 261 253 296 294 251
OEM & Other 88 78 90 73 66 77 84 73
TOTAL $30,040 $26,044 $22,103 $18,120 $13,507 $7,192 $6,051 $5,931
NVIDIA QUARTERLY REVENUE TREND
REVENUE BY MARKETS


## **2. Split Text**
We'll use RecursiveCharacterTextSplitter to break down the large text bodies from the PDFs into manageable chunks.

### 2.1 Text Splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)

# Split text into chunks for Nvidia pages
nvidia_text_chunks = []
for page in nvidia_pages:
    chunks = text_splitter.split_text(page.page_content)
    nvidia_text_chunks.extend(chunks)



### 2.2 Add Metadata

In [ ]:


# Example metadata management (customize as needed)
def add_metadata(chunks, doc_title):
    metadata_chunks = []
    for chunk in chunks:
        metadata = {
            "title": doc_title,
            "author": "company",  # Update based on document data
            "date": str(datetime.date.today())
        }
        metadata_chunks.append({"text": chunk, "metadata": metadata})
    return metadata_chunks

# Add metadata to Nvidia chunks
nvidia_chunks_with_metadata = add_metadata(nvidia_text_chunks, "NVIDIA Financial Report")



## **3. Create Embedding from text chunks**

In [ ]:
!ollama pull nomic-embed-text:v1.5

In [14]:
! ollama list

NAME                 	ID          	SIZE  	MODIFIED     
nomic-embed-text:v1.5	0a109f422b47	274 MB	38 hours ago	
llama3:latest        	365c0bd3c000	4.7 GB	38 hours ago	


In [20]:
import ollama

# Function to generate embeddings for text chunks
def generate_embeddings(text_chunks, model_name='nomic-embed-text:v1.5'):
    embeddings = []
    for chunk in text_chunks:
        # Generate the embedding for each chunk
        embedding = ollama.embeddings(model=model_name, prompt=chunk)
        embeddings.append(embedding)
    return embeddings

## Example

In [ ]:
# Example: Embed Nvidia text chunks
nvidia_texts = [chunk["text"] for chunk in nvidia_chunks_with_metadata]
nvidia_embeddings = generate_embeddings(nvidia_texts)

nvidia_embeddings

## **4. Store and Use Embeddings in Chroma DB**
After generating the embeddings, you can store them in Chroma DB for efficient retrieval

### **CHROMADB**

In [22]:
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain_community.embeddings import OllamaEmbeddings

# Wrap Nvidia texts with their respective metadata into Document objects
nvidia_documents = [Document(page_content=chunk['text'], metadata=chunk['metadata']) for chunk in nvidia_chunks_with_metadata]


# Add Nvidia embeddings to the database
nvidia_vector_db = Chroma.from_documents(documents=nvidia_documents,
                      embedding=OllamaEmbeddings(model="nomic-embed-text:v1.5",show_progress=False),
                      collection_name="nvidia-local-rag")

C:\Users\ili\AppData\Local\Temp\ipykernel_3468\1475104031.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text:v1.5",show_progress=False),


## **5. Query Processing Multi-Query Retriever:**

Implement a multi-query retriever using Chroma DB. Fetch the most relevant chunks from the database based on user queries.

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# LLM from Ollama
local_model = "llama3:latest"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
                                          nvidia_vector_db.as_retriever(),
                                          ChatOllama(model=local_model),
                                          prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from IPython.display import Markdown
questions = '''What are the main revenue drivers for Nvidia this fiscal year?'''
display(Markdown(chain.invoke(questions)))

Based on the provided financial report, the main revenue drivers for Nvidia in Q2 Fiscal 2025 are:

1. Compute & Networking segment, which accounted for $26,446 million (up 17% from Q1 FY25 and up 154% from Q2 FY24).
2. Data Center market platform, which contributed $26,272 million (up 16% from Q1 FY25 and up 154% from Q2 FY24).
3. Compute market platform, which generated $22,604 million (up 17% from Q1 FY25 and up 162% from Q2 FY24).

These segments and platforms drove the company's overall revenue growth of 15% year-over-year to $30,040 million in Q2 Fiscal 2025.

In [ ]:
questions = '''Can you some financial advise on Nvidia Stock to the future? should people consider buying it?'''
display(Markdown(chain.invoke(questions)))

Based only on the provided context (NVIDIA Financial Report), here's my analysis:

**Revenue Growth:** NVIDIA has consistently shown strong revenue growth, with a significant increase in the current quarter ($32.5 billion expected in Q3 2025). This suggests that the company is expanding its market presence and diversifying its product offerings.

**Gross Margin:** The gross margin expectations (74.4% GAAP and 75.0% non-GAAP) indicate that NVIDIA is maintaining a healthy profit margin, which is positive for investors.

**Operating Expenses:** While operating expenses are expected to grow in the mid-to-upper 40% range, this is still relatively moderate compared to revenue growth. This could suggest that NVIDIA is investing in its business to drive future growth.

**Cash Flow:** The company has generated significant cash flow from operations ($14.5 billion in Q2 2024) and has a strong balance sheet with $34.8 billion in cash, cash equivalents, and marketable securities.

**Share Repurchases and Dividends:** NVIDIA has been returning value to shareholders through share repurchases ($7.2 billion in Q2 2024) and increasing its dividend payout (150% increase in Q2 2024).

Considering these factors, here are some points to ponder for potential investors:

1. **Growth Potential:** If you believe in the growth of the industries NVIDIA serves (AI, gaming, cloud computing), the company's strong financials and growth prospects could make it an attractive investment.
2. **Valuation:** Keep in mind that NVIDIA's stock price has historically been volatile. Evaluate the company's valuation relative to its peers and consider the potential risks and rewards.
3. **Industry Trends:** Monitor industry trends, such as the adoption of AI and cloud computing, to gauge the demand for NVIDIA's products and services.
4. **Competition:** Assess the competitive landscape in NVIDIA's key markets, including the rise of alternative GPU vendors and competition from emerging technologies.

In conclusion, while no investment is without risk, NVIDIA's strong financials, revenue growth, and cash flow generation make it an attractive option for those willing to take on some market volatility. However, always remember to evaluate the company's valuation, industry trends, and competitive landscape before making a decision.

# II. Chatting with Local RAG - Hugging Face Embedding + Llama 3 -> Improve Speed

In [16]:
from langchain_community.llms.ollama import Ollama


local_model = "llama3:latest"
cached_llm = Ollama(model=local_model)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

# Load a smaller Hugging Face embedding model
#hf_embedding_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

# Wrap the Hugging Face model for use with LangChain
#hf_embeddings = HuggingFaceEmbeddings(model=SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2'), show_progress=True)

# Wrap Nvidia texts with their respective metadata into Document objects
nvidia_documents = [Document(page_content=chunk['text'], metadata=chunk['metadata']) for chunk in nvidia_chunks_with_metadata]

model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Add Nvidia embeddings to the database using the smaller Hugging Face model
nvidia_vector_db_hf = Chroma.from_documents(documents=nvidia_documents,
                      embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
                      collection_name="nvidia-local-rag-384")
